In [10]:
import numpy as np
import pandas as pd
import itertools
import pickle
import sys
import os
import shutil
import tempfile
import subprocess
import dsc
from dsc.query_engine import Query_Processor as dscQP
from dsc import dsc_io

import matplotlib.pyplot as plt
from pymir import mpl_stylesheet
from pymir import mpl_utils

mpl_stylesheet.banskt_presentation(splinecolor = 'black', dpi = 120)
from scipy.spatial import procrustes

In [3]:
dsc_output = "/gpfs/commons/groups/knowles_lab/sbanerjee/low_rank_matrix_approximation_numerical_experiments/lrma_single"

dsc_fname  = os.path.basename(os.path.normpath(dsc_output))
db = os.path.join(dsc_output, dsc_fname + ".db")
respkl   = os.path.join("../dsc/results", dsc_fname + ".pkl")
dscoutpkl = os.path.join("../dsc/results", dsc_fname + "_dscout.pkl")

if os.path.isfile(dscoutpkl):
    dscout = pd.read_pickle(dscoutpkl)
else:
    print ("Could not find dscout from dscquery")

refresh_pickle = True
sim_module = "blockdiag"
target = [sim_module] + [f"{sim_module}.{prop}" for prop in ["p", "k", "h2", "h2_shared_frac", "aq", "a0"]]

condition = [""]

print ("Reading from DSC database:")
print (f"    {db}")
print ("")

qp = dscQP(db, target, condition)
df = qp.output_table

df

Could not find dscout from dscquery
Reading from DSC database:
    /gpfs/commons/groups/knowles_lab/sbanerjee/low_rank_matrix_approximation_numerical_experiments/lrma_single/lrma_single.db



,DSC,blockdiag.output.file,blockdiag.p,blockdiag.k,blockdiag.h2,blockdiag.h2_shared_frac,blockdiag.aq,blockdiag.a0
0,1,blockdiag/blockdiag_1,1000,100,0.6,0.6,0.6,0.2


In [4]:
#fprefix = df.loc[df['DSC'] == 2]['blockdiag_p.output.file'].values[0]
fprefix = df.iloc[[0]][f'{sim_module}.output.file'].values[0]
fname = os.path.join(dsc_output, f"{fprefix}.pkl")
data  = dsc_io.load_dsc(fname)

In [5]:
Z = data['Z']
Z_cent = Z - np.mean(Z, axis = 0, keepdims = True)
labels = np.array(data['Ctrue'])

# Run FactorGO on `Z_cent`

In [8]:
import tempfile
os_handle, fgo_dir = tempfile.mkdtemp()

In [14]:
zscore_df = pd.DataFrame(Z_cent.T)
zscore_df.columns = [f"trait{x+1}" for x in range(Z_cent.shape[0])]
zscore_df.insert(0, 'rsid', [f"rs{x + 1}" for x in range(Z_cent.shape[1])])
zscore_fname = os.path.join(fgo_dir, "zscore.tsv")
zscore_df.to_csv(fname, sep = '\t', index = False)

In [24]:
N_fname = os.path.join(fgo_dir, "sampleN.tsv")
np.savetxt(N_fname, np.repeat(10000, Z_cent.shape[0]), fmt='%d', header = 'N')

In [25]:
fgo_res_prefix = os.path.join(fgo_dir, "simres")
W_mean = np.loadtxt(f"{fgo_res_prefix}.Wm.tsv.gz")
W_var  = np.loadtxt(f"{fgo_res_prefix}.Wvar.tsv.gz")
Z_mean = np.loadtxt(f"{fgo_res_prefix}.Zm.tsv.gz")
Z_var  = np.loadtxt(f"{fgo_res_prefix}.Zvar.tsv.gz")
factor_info = np.loadtxt(f"{fgo_res_prefix}.factor.tsv.gz")
factor_isort = np.array([int(x) for x in factor_info[:, 0]])
factor_mean_ard = factor_info[:, 1]
factor_variance_explained = factor_info[:, 2]

In [39]:
factor_info[:, 1]

array([  75267.04952945,   88815.08775464, 1168297.99321385,
       1218985.52558135, 1313021.2751539 ])

In [40]:
import shutil
if os.path.isdir(fgo_dir):
    shutil.rmtree(fgo_dir)

In [24]:
n_factor = 5

fgo_dir = tempfile.mkdtemp()

# Step 1. Save data in tsv
zscore_df = pd.DataFrame(Z_cent.T)
zscore_df.columns = [f"z{x+1}" for x in range(Z_cent.shape[0])]
zscore_df.insert(0, 'rsid', [f"rs{x + 1}" for x in range(Z_cent.shape[1])])
zscore_fname = os.path.join(fgo_dir, "zscore.tsv")
zscore_df.to_csv(zscore_fname, sep = '\t', index = False)

# Step 2. Save NSamples in tsv
N_fname = os.path.join(fgo_dir, "sampleN.tsv")
np.savetxt(N_fname, np.repeat(10000, Z_cent.shape[0]), fmt='%d', header = 'N')

# Step 3. Run factorgo
fgo_res_prefix = os.path.join(fgo_dir, "simres")
cmd = ["factorgo"]
cmd += [zscore_fname, N_fname]
cmd += ["-k", f"{n_factor}"]
cmd += ["-o", fgo_res_prefix]
process = subprocess.run(cmd, 
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE)

# Step 4. Collect results
W_mean = np.loadtxt(f"{fgo_res_prefix}.Wm.tsv.gz")
W_var  = np.loadtxt(f"{fgo_res_prefix}.Wvar.tsv.gz")
Z_mean = np.loadtxt(f"{fgo_res_prefix}.Zm.tsv.gz")
Z_var  = np.loadtxt(f"{fgo_res_prefix}.Zvar.tsv.gz")
factor_info = np.loadtxt(f"{fgo_res_prefix}.factor.tsv.gz")
factor_isort = np.array([int(x) for x in factor_info[:, 0]])
factor_mean_ard = factor_info[:, 1]
factor_variance_explained = factor_info[:, 2]

# Step 5. Delete temp_dir
if os.path.isdir(fgo_dir):
    shutil.rmtree(fgo_dir)

In [29]:
factor_variance_explained

array([0.16097099, 0.13429882, 0.00941938, 0.00923971, 0.00855748])

In [31]:
vars(process)

{'args': ['factorgo',
  '/scratch/tmp1pjxa0sh/zscore.tsv',
  '/scratch/tmp1pjxa0sh/sampleN.tsv',
  '-k',
  '5',
  '-o',
  '/scratch/tmp1pjxa0sh/simres'],
 'returncode': 0,
 'stdout': b'',
 'stderr': b'WARNING: All log messages before absl::InitializeLog() is called are written to STDERR\nI0000 00:00:1705363998.691059   39410 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.\n[2024-01-15 19:13:18 - INFO] \n             #############################################\n\n                   Welcome to use FactorGo!\n                   Version: 1.0.0\n\n             #############################################\n                                                           \n[2024-01-15 19:13:18 - INFO] Loading GWAS summary statistics and sample size.\n[2024-01-15 19:13:19 - INFO] Finished loading GWAS summary statistics and sample size.\n[2024-01-15 19:13:19 - INFO] Found N = 500 studies, P = 1000 SNPs\n[2024-01-15 19:13:19 - INFO] User set K = 5 latent factors.\n[2024-01-15 19:13:19 - INFO] 

In [32]:
data['Ltrue'].shape

(500, 100)

In [20]:
print (process.returncode)
print (process.stderr.decode('utf-8'))

0
I0000 00:00:1705363913.620576   39045 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
[2024-01-15 19:11:53 - INFO] 
             #############################################

                   Welcome to use FactorGo!
                   Version: 1.0.0

             #############################################
                                                           
[2024-01-15 19:11:53 - INFO] Loading GWAS summary statistics and sample size.
[2024-01-15 19:11:54 - INFO] Finished loading GWAS summary statistics and sample size.
[2024-01-15 19:11:54 - INFO] Found N = 500 studies, P = 1000 SNPs
[2024-01-15 19:11:54 - INFO] User set K = 5 latent factors.
[2024-01-15 19:11:54 - INFO] set hyper parameters
            halpha_a: 1e-05,
            halpha_b: 1e-05,
            htau_a: 1e-05,
            htau_b: 1e-05,
            hbeta: 1e-05
        
[2024-01-15 19:11:54 - INFO] Initializing mean parameters with seed 123456789.
[2024-01-15 19:11:54 - INFO] Completed initialization.

In [ ]:
    def rds_wrapper(self, X, y, sk, binit, wkinit, s2init, maxiter, epstol, convtol,
                    update_pi = True, update_sigma2 = True):
        os_handle, data_rds_file = tempfile.mkstemp(suffix = ".rds")
        datadict = {'X': X, 'y': y, 'sk2': np.square(sk), 
                    'binit': binit, 'winit': wkinit, 's2init': s2init}
        R_utils.save_rds(datadict, data_rds_file)
        os_handle, out_rds_file = tempfile.mkstemp(suffix = ".rds")
        cmd  = ["Rscript",   self.rscript_file]
        cmd += ["--outfile", out_rds_file]
        cmd += ["--infile",  data_rds_file]
        cmd += ["--maxiter", f"{maxiter}"]
        cmd += ["--epstol",  f"{epstol}"]
        cmd += ["--convtol", f"{convtol}"]
        if not update_pi:     cmd += ["--fix_pi"]
        if not update_sigma2: cmd += ["--fix_sigma2"]

        process = subprocess.Popen(cmd,
                                   stdout = subprocess.PIPE,
                                   stderr = subprocess.PIPE
                                  )
        res     = process.communicate()
        self.logger.info(res[0].decode('utf-8'))
    
        if len(res[1].decode('utf-8')) > 0:
            self.logger.debug("ERROR ==>")
            self.logger.debug(res[1].decode('utf-8'))

        retcode  = process.returncode
        fit_dict = R_utils.load_rds(out_rds_file) if retcode == 0 else None
        if os.path.exists(data_rds_file): os.remove(data_rds_file)
        if os.path.exists(out_rds_file):  os.remove(out_rds_file)
        return fit_dict